In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import geojson
import firedrake
import icepack, icepack.plot, icepack.models

In [ ]:
outline_filename = icepack.datasets.fetch_helheim_outline()
with open(outline_filename, 'r') as outline_file:
    outline = geojson.load(outline_file)

geometry = icepack.meshing.collection_to_geo(outline, lcar=5e3)
with open('helheim.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
!gmsh -2 -format msh2 -v 2 -o helheim.msh helheim.geo

In [ ]:
mesh = firedrake.Mesh('helheim.msh')

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh, axes=axes)
axes.legend()

In [ ]:
thickness_filename = icepack.datasets.fetch_bedmachine_greenland()
thickness = rasterio.open('netcdf:' + thickness_filename + ':thickness', 'r')
surface = rasterio.open('netcdf:' + thickness_filename + ':surface', 'r')

Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
h0 = icepack.interpolate(thickness, Q)
s0 = icepack.interpolate(surface, Q)

from firedrake import inner, grad, dx
def smooth(q0, α):
    q = q0.copy(deepcopy=True)
    J = 0.5 * ((q - q0)**2 + α**2 * inner(grad(q), grad(q))) * dx
    F = firedrake.derivative(J, q)
    firedrake.solve(F == 0, q)
    return q

α = firedrake.Constant(2e3)
h = smooth(h0, α)
s = smooth(s0, α)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(h, 40, axes=axes)
fig.colorbar(contours)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(s, 40, axes=axes)
fig.colorbar(contours)

In [ ]:
velocity_filenames = icepack.datasets.fetch_measures_greenland()
vx_filename = [f for f in velocity_filenames if 'vx' in f][0]
vy_filename = [f for f in velocity_filenames if 'vy' in f][0]
vx = rasterio.open(vx_filename, 'r')
vy = rasterio.open(vy_filename, 'r')

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)
u_obs = icepack.interpolate((vx, vy), V)

In [ ]:
from firedrake import grad
from icepack.constants import ice_density as ρ_I, gravity as g
τ = firedrake.project(-ρ_I * g * h * grad(s), V)

In [ ]:
import numpy as np
fig, axes = icepack.plot.subplots()
levels = np.linspace(0, .5, 51)
contours = icepack.plot.tricontourf(τ, levels=levels, extend='max', axes=axes)
fig.colorbar(contours)

In [ ]:
u = u_obs.copy(deepcopy=True)
from firedrake import sqrt
U = sqrt(inner(u, u))
Σ = sqrt(inner(τ, τ))
C = firedrake.interpolate(0.5 * Σ / U**(1/3), Q)

In [ ]:
T = firedrake.Constant(260.)
A = icepack.rate_factor(T)

In [ ]:
ice_stream = icepack.models.IceStream()
opts = {'dirichlet_ids': [1, 5, 6], 'side_wall_ids': [2, 4], 'tol': 1e-6}
u = ice_stream.diagnostic_solve(u0=u_obs, h=h, s=s, A=A, C=C, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(u, axes=axes)
fig.colorbar(contours)